This is the logistic iterations formula: 

$$
x_{n + 1} = rx_{n}(1 - x_n)
$$

The system experience Chaos for certain value of R, find that and plot the infinite bifurcations graphs. 

In [3]:
# An identity map
TheDic = Dict{Number, Number}(zip(1:10, 1:10))

Dict{Number,Number} with 10 entries:
  7  => 7
  4  => 4
  9  => 9
  10 => 10
  2  => 2
  3  => 3
  5  => 5
  8  => 8
  6  => 6
  1  => 1